In [35]:
import numpy as np
import pandas as pd
import pickle
import sklearn
from sklearn import datasets
from feature_extraction import get_names
from pag import pag

# Load data

In [40]:
iris = datasets.load_iris()
X_iris = iris.data
y_iris = iris.target

In [53]:
# Read musical features
infile = open('dataset_features.pckl','rb')
features = pickle.load(infile)
infile.close()

In [87]:
names = get_names()
names.append("Label")
df = pd.DataFrame(features, columns = names)
list_cols = []
for coll in df.columns:
  col = str(coll)
  if(col.find("pitch") > 0 and (col.find("min") > 0 or col.find("max") > 0)): 
    continue
  if(col in ["tempogram_min", "tempogram_max"]):
    continue
  list_cols.append(col)
print(len(list_cols))
df_songs = df[list_cols]
df_songs.head()

119


,1_pitch_mean,1_pitch_std,2_pitch_mean,2_pitch_std,3_pitch_mean,3_pitch_std,4_pitch_mean,4_pitch_std,5_pitch_mean,5_pitch_std,...,18_mfcc_std,19_mfcc_min,19_mfcc_max,19_mfcc_mean,19_mfcc_std,20_mfcc_min,20_mfcc_max,20_mfcc_mean,20_mfcc_std,Label
0,0.367158,0.250658,0.446059,0.303279,0.345249,0.267387,0.378227,0.330485,0.358498,0.265262,...,6.322423,-35.063286,22.445841,-6.158485,7.843939,-15.309130,32.983795,5.411145,6.995747,0.0
1,0.548021,0.321238,0.505586,0.274560,0.521004,0.300331,0.554739,0.276167,0.581093,0.289333,...,7.282372,-24.618061,21.180487,-0.404446,5.324600,-18.628052,39.097198,3.593739,5.749859,0.0
2,0.228689,0.281790,0.141832,0.164917,0.200010,0.270996,0.226451,0.255719,0.352861,0.332630,...,7.605660,-29.139126,22.793182,-1.253622,7.343888,-26.660023,21.146843,-0.899938,6.930117,0.0
3,0.341178,0.246983,0.478602,0.343246,0.281382,0.211076,0.306389,0.270918,0.313350,0.240436,...,5.671211,-28.038145,19.897411,-5.406955,5.860555,-18.276693,26.443262,1.132697,6.015408,0.0
4,0.227324,0.213584,0.520437,0.396417,0.204974,0.203153,0.249976,0.289113,0.297624,0.357263,...,7.713253,-32.549755,21.090767,-4.515356,8.532988,-27.717663,29.424013,-4.685843,8.793524,0.0


In [88]:
epsilons = [0.1, 0.2, 0.3]
deltas = [0.1, 0.2, 0.3]
num_features = 10
depth_tree = 3
models = ["Logistic Regression",  "Decision Tree", "SVM linear", "SVM polynomial", "SVM rbf"]
n_results = []
for delta,epsilon in zip(deltas,epsilons):
  ns = ['('+str(delta)+','+str(epsilon)+')']
  for model in models:
    ns.append(pag(epsilon = epsilon, delta = delta, model = model,  num_features = 118, depth_tree = 3))
  n_results.append(ns)


In [89]:
df_pags = pd.DataFrame(n_results)
df_pags.columns = ["delta/epsilon","Logistic Regression",  "Decision Tree", "SVM linear", "SVM polynomial", "SVM rbf"]
df_pags.head()

,delta/epsilon,Logistic Regression,Decision Tree,SVM linear,SVM polynomial,SVM rbf
0,"(0.1,0.1)",70.817086,2026.800009,70.732697,40.943446,inf
1,"(0.2,0.2)",31.942807,500.694340,31.900613,17.005987,inf
2,"(0.3,0.3)",19.943654,220.969445,19.915525,9.985774,inf
